In [4]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [6]:
#Loading dataset from tensorflow
mnist_dataset,mnist_info=tfds.load(name="mnist",with_info=True,as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to C:\Users\vivek jha\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [7]:
#Splitting into train and test
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [18]:
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)

test_data=mnist_test.map(scale)

buffer_size=10000

shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size=100

train_data=train_data.batch(batch_size)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs,validation_targets=next(iter(validation_data))

In [27]:
#Model
input_size=784
output_size=10
hidden_layer=100

model=tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                        ])

In [28]:
#Optimizer and Loss function

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [29]:
#Training

num_epochs=5

model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),validation_steps=10,verbose=2)

Epoch 1/5
540/540 - 12s - loss: 0.3375 - accuracy: 0.9036 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 12s - loss: 0.1407 - accuracy: 0.9584 - val_loss: 0.1139 - val_accuracy: 0.9668
Epoch 3/5
540/540 - 11s - loss: 0.0990 - accuracy: 0.9706 - val_loss: 0.0974 - val_accuracy: 0.9708
Epoch 4/5
540/540 - 12s - loss: 0.0776 - accuracy: 0.9765 - val_loss: 0.0780 - val_accuracy: 0.9755
Epoch 5/5
540/540 - 11s - loss: 0.0624 - accuracy: 0.9811 - val_loss: 0.0662 - val_accuracy: 0.9780


In [30]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] 0.0882 - accuracy: 0.97 - 4s 4s/step - loss: 0.0882 - accuracy: 0.9729


In [32]:
test_accuracy

0.9729